In [1]:
import os
import numpy as np
import efficientnet.tfkeras
from tensorflow.keras.models import load_model
from tensorflow.keras import models
import tensorflow as tf
from focal_loss import BinaryFocalLoss

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [3]:
model_dir ='/media/tohn/SSD/ModelTrainByImages/R2_1/models/B5R2b5_AN_1FC_fold1_1.h5'
model = load_model(model_dir)
height = width = model.input_shape[1]

x = model.get_layer('head_pooling').output
prediction_layer = model.output
model2 = models.Model(inputs= model.input, outputs=[prediction_layer,x]) 

In [4]:
#ไม่ต้องเปลี่ยนก็ได้ เพราะเอา Label มาใช้เฉยๆ 
# validation
import pandas as pd
base_dir  = '/media/tohn/SSD/Images/Image1/validation/'
dataframe = pd.read_csv( '/home/yupaporn/codes/USAI/Validationdf_fold1_3.csv')
validation_dir = os.path.join(base_dir, 'validation')

#Train
train_df = pd.read_csv( '/home/yupaporn/codes/USAI/Traindf_fold1_3.csv')
base_dir0 = '/media/tohn/SSD/Images/Image1/'
os.chdir(base_dir0)
train_dir = os.path.join(base_dir0, 'train')

In [5]:
import pandas as pd
df0 = pd.read_csv (r'/home/yupaporn/codes/USAI/Validationdf_fold1.csv')
print(df0 .shape)
dataframe = df0[(df0['Path Crop']!='None' )&(df0['Path Crop']!='Nan')]
# a = df01[df01['Case'].between(1, 10)]
# dataframe = df01[df01['Abs Position']!='P8']
print(dataframe.shape)
print('Normal: ',dataframe[dataframe['Class']=='Normal'].shape)
print('Abnormal: ',dataframe[dataframe['Class']=='Abnormal'].shape)
dataframe.head(5)

(656, 22)
(656, 22)
Normal:  (429, 22)
Abnormal:  (227, 22)


,Unnamed: 0,Unnamed: 0.1,Case,Abs Position,Sub Position,Class,Sub_class,Path Full,Path Crop,Views,...,originalImage,left,top,width,height,Rleft,Rtop,Rwidth,Rheight,filename
0,0,0,15,P3,P31,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-B,...,https://irisprodseatraining.blob.core.windows....,35.0,96.0,694.0,584.0,0.036832,0.107041,0.712707,0.649402,AB01 P3-1 C015.JPG
1,1,1,15,P1,P1,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-A,...,https://irisprodseatraining.blob.core.windows....,141.0,96.0,586.0,614.0,0.143898,0.107041,0.595628,0.683267,AB01 P1 C015.JPG
2,2,2,15,P2,P2,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-A,...,https://irisprodseatraining.blob.core.windows....,242.0,101.0,491.0,451.0,0.245009,0.113017,0.497278,0.501992,AB01 P2 C015.JPG
3,3,3,15,P4,P42,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-B,...,https://irisprodseatraining.blob.core.windows....,154.0,110.0,714.0,611.0,0.155235,0.122977,0.718412,0.679283,AB01 P4-2 C015.JPG
4,4,4,15,P4,P41,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-B,...,https://irisprodseatraining.blob.core.windows....,152.0,146.0,609.0,519.0,0.152057,0.162817,0.608229,0.577689,AB01 P4-1 C015.JPG


In [6]:
# dataframe.info()

In [6]:
batch_size = 64
epochs = 10

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      brightness_range=[0.5,1.5],
      shear_range=0.4,
      zoom_range=0.2,
      horizontal_flip=False,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(
        dataframe = dataframe,
        directory = train_dir,
        x_col = 'Path Crop',
        y_col = 'Sub_class',
        target_size = (height, width),
        batch_size=batch_size,
        color_mode= 'rgb',
        class_mode='categorical')

#label
labels = (train_generator.class_indices)
labels = dict((v,k.replace("C","")) for k,v in labels.items())
print(labels)

Found 656 validated image filenames belonging to 15 classes.
{0: 'AB01', 1: 'AB02', 2: 'AB03', 3: 'AB04', 4: 'AB05', 5: 'AB06', 6: 'AB07', 7: 'AB081', 8: 'AB082', 9: 'AB083', 10: 'AB09', 11: 'AB10', 12: 'AB11', 13: 'AB12', 14: 'Normal'}


In [7]:
from tensorflow.keras.preprocessing import image
def predict_image(img_path):
    # Read the image and resize it
    img = image.load_img(img_path, target_size=(height, width))
    # Convert it to a Numpy array with target shape.
    x = image.img_to_array(img)
    # Reshape
    x = x.reshape((1,) + x.shape)
    x /= 255.
    result = model2.predict([x])
    result1 = result[0][0]
    result2 = result[1][0]

    return result1, result2

#Predict
pred_list = list()
prob_list = list()
vectorlist = list()
img_path=dataframe['Path Crop'].tolist()
for i in range(0,len(img_path)):
    result1,result2 = predict_image(img_path[i])
    result = np.argmax(result1)
    pred_list.append(labels[result])
    prob_list.append(result1[result])
    vectorlist.append(result2.tolist())
    
dataframe['category'] = pred_list
dataframe['Prob'] = prob_list

In [8]:
vector_array = np.array(vectorlist)
vector_array

array([[-0.15373729, -0.13878427,  0.1379433 , ..., -0.06146726,
        -0.12220816, -0.09252419],
       [-0.02843768, -0.133757  ,  0.28213012, ..., -0.06311013,
        -0.07857281, -0.08835533],
       [-0.10715667, -0.13064463,  0.10954557, ..., -0.10252369,
        -0.15461358, -0.05655931],
       ...,
       [ 1.28748798,  1.44685519, -0.11500701, ...,  0.96426868,
        -0.09894986,  0.90301526],
       [ 1.06654549,  1.38069689, -0.20380035, ...,  1.45312905,
        -0.18035719,  1.10180986],
       [ 0.0990036 ,  0.39985123, -0.19624032, ...,  0.36286744,
        -0.07489748,  0.40198562]])

In [9]:
dataset = pd.DataFrame(vector_array[:, :])
dataset

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,-0.153737,-0.138784,0.137943,-0.123214,-0.206309,-0.188063,0.100369,-0.049404,-0.180050,-0.127577,...,-0.122875,-0.201280,-0.205665,-0.122383,-0.137319,0.103782,-0.105585,-0.061467,-0.122208,-0.092524
1,-0.028438,-0.133757,0.282130,-0.108657,-0.210327,-0.043263,0.389655,-0.188884,-0.079164,0.065990,...,-0.117456,-0.074096,-0.204521,-0.062230,-0.209019,0.005034,-0.102927,-0.063110,-0.078573,-0.088355
2,-0.107157,-0.130645,0.109546,0.023668,-0.181899,-0.125587,0.432066,-0.194315,-0.183115,0.057802,...,-0.100638,0.004042,-0.227944,-0.041898,-0.193133,-0.073300,-0.142099,-0.102524,-0.154614,-0.056559
3,-0.144092,-0.144814,0.306699,-0.005712,-0.203596,-0.051891,0.207564,-0.127166,-0.193819,0.128082,...,-0.076378,-0.067315,-0.213885,-0.100161,-0.196189,0.186741,-0.079633,-0.127044,-0.037541,-0.149690
4,-0.054945,-0.051653,0.251870,-0.101658,-0.136217,-0.134739,0.257016,-0.099523,-0.084570,-0.038034,...,-0.052175,-0.136952,-0.209007,-0.045299,-0.050552,-0.089773,-0.091959,-0.064067,-0.109136,-0.148610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651,0.764140,1.423839,0.001750,-0.097308,0.627257,-0.137619,-0.155140,0.481085,0.984165,-0.140271,...,0.780579,-0.094969,0.489646,0.809451,0.051397,-0.168722,-0.156749,1.168075,-0.161617,0.825065
652,0.389699,0.051683,-0.127803,-0.133451,0.046052,-0.111284,-0.052574,0.191022,0.024296,-0.080769,...,0.302034,0.138480,-0.066647,0.206239,0.215155,-0.065612,-0.137907,0.110669,-0.137874,0.153426
653,1.287488,1.446855,-0.115007,-0.134472,0.624818,-0.159727,-0.063722,0.743706,0.442402,-0.035285,...,0.742781,-0.189759,0.883532,0.884573,0.647148,-0.127578,-0.183933,0.964269,-0.098950,0.903015
654,1.066545,1.380697,-0.203800,-0.143413,0.569574,-0.069633,-0.091297,0.828280,1.114043,-0.145413,...,1.244310,-0.197012,0.868451,1.011054,0.471728,-0.181323,-0.188003,1.453129,-0.180357,1.101810


In [10]:
a = dataframe['Class'].tolist()
b = dataframe['Sub Position'].tolist()
c = dataframe['Views'].tolist()
d = dataframe['Sub_class'].tolist()
dataset['Class'] = a 
dataset['SubPosition'] = b
dataset['Views'] = c
dataset['Sub_class'] = d

In [11]:
dataset

,0,1,2,3,4,5,6,7,8,9,...,2042,2043,2044,2045,2046,2047,Class,SubPosition,Views,Sub_class
0,-0.153737,-0.138784,0.137943,-0.123214,-0.206309,-0.188063,0.100369,-0.049404,-0.180050,-0.127577,...,-0.137319,0.103782,-0.105585,-0.061467,-0.122208,-0.092524,Abnormal,P31,FP-B,AB01
1,-0.028438,-0.133757,0.282130,-0.108657,-0.210327,-0.043263,0.389655,-0.188884,-0.079164,0.065990,...,-0.209019,0.005034,-0.102927,-0.063110,-0.078573,-0.088355,Abnormal,P1,FP-A,AB01
2,-0.107157,-0.130645,0.109546,0.023668,-0.181899,-0.125587,0.432066,-0.194315,-0.183115,0.057802,...,-0.193133,-0.073300,-0.142099,-0.102524,-0.154614,-0.056559,Abnormal,P2,FP-A,AB01
3,-0.144092,-0.144814,0.306699,-0.005712,-0.203596,-0.051891,0.207564,-0.127166,-0.193819,0.128082,...,-0.196189,0.186741,-0.079633,-0.127044,-0.037541,-0.149690,Abnormal,P42,FP-B,AB01
4,-0.054945,-0.051653,0.251870,-0.101658,-0.136217,-0.134739,0.257016,-0.099523,-0.084570,-0.038034,...,-0.050552,-0.089773,-0.091959,-0.064067,-0.109136,-0.148610,Abnormal,P41,FP-B,AB01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651,0.764140,1.423839,0.001750,-0.097308,0.627257,-0.137619,-0.155140,0.481085,0.984165,-0.140271,...,0.051397,-0.168722,-0.156749,1.168075,-0.161617,0.825065,Normal,P32,FP-B,Normal
652,0.389699,0.051683,-0.127803,-0.133451,0.046052,-0.111284,-0.052574,0.191022,0.024296,-0.080769,...,0.215155,-0.065612,-0.137907,0.110669,-0.137874,0.153426,Normal,P42,FP-B,Normal
653,1.287488,1.446855,-0.115007,-0.134472,0.624818,-0.159727,-0.063722,0.743706,0.442402,-0.035285,...,0.647148,-0.127578,-0.183933,0.964269,-0.098950,0.903015,Normal,P52,FP-C,Normal
654,1.066545,1.380697,-0.203800,-0.143413,0.569574,-0.069633,-0.091297,0.828280,1.114043,-0.145413,...,0.471728,-0.181323,-0.188003,1.453129,-0.180357,1.101810,Normal,P61,FP-C,Normal


In [12]:
os.chdir( '/home/yupaporn/codes/USAI/')
testdf = dataset
testdf.to_csv('FVvalidation_AN_RF_15AB_fold1_1.csv')